In [1]:
import numpy as np
import arcode
import codec
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
'''
pref = './Paper_RB_14bit_COMPINTs/CI884/COMPINT'
nx=256
ny=480
ncomp=4
'''

'''
pref = './Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=240
ncomp=4
'''


pref = './Paper_RB_14bit_COMPINTs/CI32324/COMPINT'
nx=64
ny=120
ncomp=4


In [7]:
i_f = 3
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 64, 120, 4) , mean = 128.11826171875 , std = 11.098583681659338 , Max = 180.0 , Min = 83.0 , type=float64
t_C_0 shape: (1, 64, 120) , mean = 114.86015625 , std = 5.702861669862416 , Max = 123.0 , Min = 83.0 , type=float64
t_C_1 shape: (1, 64, 120) , mean = 136.2203125 , std = 2.3512305967323615 , Max = 167.0 , Min = 132.0 , type=float64
t_C_2 shape: (1, 64, 120) , mean = 139.68971354166666 , std = 5.193690926381253 , Max = 180.0 , Min = 123.0 , type=float64
t_C_3 shape: (1, 64, 120) , mean = 121.70286458333334 , std = 3.3485059047524954 , Max = 127.0 , Min = 101.0 , type=float64


In [8]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (64, 120) , mean = -6.297135416666666 , std = 3.3485059047524954 , Max = -1 , Min = -27 , type=int8


In [9]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen7680
size17692
size23523
ratio1= 1.0015625
ratio2= 0.458723958333


In [10]:
fs = 1; fe = 4;
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        idx = i-fs
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
    
    print('\n')

1
  t_C shape: (1, 64, 120, 4) , mean = 128.00716145833334 , std = 16.57902126512141 , Max = 186.0 , Min = 85.0 , type=float64
 channel0 shape: (64, 120) , mean = -14.781510416666666 , std = 8.641178775798366 , Max = 5 , Min = -41 , type=int8
   olen: 7680 Bytes
   size1: 7692 Bytes
   size2: 4594 Bytes
ratio1= 1.0015625
ratio2= 0.598177083333
 channel1 shape: (64, 120) , mean = 12.663802083333334 , std = 4.05380378502656 , Max = 32 , Min = 1 , type=int8
   olen: 7680 Bytes
   size1: 7692 Bytes
   size2: 3792 Bytes
ratio1= 1.0015625
ratio2= 0.49375
 channel2 shape: (64, 120) , mean = 16.640364583333334 , std = 8.859616119434614 , Max = 58 , Min = -13 , type=int8
   olen: 7680 Bytes
   size1: 7692 Bytes
   size2: 4720 Bytes
ratio1= 1.0015625
ratio2= 0.614583333333
 channel3 shape: (64, 120) , mean = -14.494010416666667 , std = 8.001055630866768 , Max = 1 , Min = -43 , type=int8
   olen: 7680 Bytes
   size1: 7692 Bytes
   size2: 4563 Bytes
ratio1= 1.0015625
ratio2= 0.594140625


2
  t_C 

In [11]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.0015625
   ratio_2= 0.5751627604166667
 
2
   ratio_1= 1.0015625
   ratio_2= 0.5595377604166667
 
3
   ratio_1= 1.0015625
   ratio_2= 0.4849934895833333
 
